In [1]:
import tkinter as tk
from tkinter import ttk
import math
from functools import cmp_to_key
import timeit

from itertools import cycle

global execution_time
execution_time = timeit.default_timer()

# Define grid parameters
GRID_SPACING = 20
GRID_COLOR = "gray"

# Define label for displaying cursor coordinates
COORDINATE_LABEL = "Cursor Coordinates: "

global execution_time
execution_time = timeit.default_timer()


class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


def orientation(p, q, r):
    val = (q.y - p.y) * (r.x - q.x) - (q.x - p.x) * (r.y - q.y)
    if val == 0:
        return 0  # collinear
    return 1 if val > 0 else -1  # clockwise or counterclockwise


def compare(p0, p1, p2):
    o = orientation(p0, p1, p2)
    if o == 0:
        if dist_sq(p0, p2) >= dist_sq(p0, p1):
            return -1
        else:
            return 1
    else:
        return -1 if o == -1 else 1


def dist_sq(p1, p2):
    return (p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2


def graham_scan(points):
    def polar_angle(point, pivot):
        return math.atan2(point.y - pivot.y, point.x - pivot.x)

    def graham_compare(p1, p2):
        angle1 = polar_angle(p1, pivot)
        angle2 = polar_angle(p2, pivot)

        if angle1 < angle2:
            return -1
        elif angle1 > angle2:
            return 1
        else:
            return -1 if dist_sq(pivot, p1) < dist_sq(pivot, p2) else 1

    def graham_scan_algorithm(points):
        start_time = timeit.default_timer()
        nonlocal pivot
        n = len(points)
        if n < 3:
            return []
        
        start_time = timeit.default_timer()
        pivot = min(points, key=lambda p: (p.y, p.x))
        sorted_points = sorted(points, key=cmp_to_key(graham_compare))

        stack = [pivot, sorted_points[0], sorted_points[1]]

        for i in range(2, n):
            while (
                len(stack) > 1
                and orientation(stack[-2], stack[-1], sorted_points[i]) != -1
            ):
                stack.pop()
            stack.append(sorted_points[i])
        global execution_time
        execution_time = (timeit.default_timer() - start_time) * 1000
        print(f"Graham Scan Execution Time: {execution_time:.6f} milliseconds")

        return stack

    pivot = None
    return graham_scan_algorithm(points)


def jarvis_march(points):
    start_time = timeit.default_timer()
    num_points = len(points)
    if num_points < 3:
        return []

    # Find the leftmost point as the starting point
    start_point = min(points, key=lambda p: (p.x, p.y))
    convex_hull_points = []
    current_point = start_point

    while True:
        convex_hull_points.append(current_point)
        next_point = None

        for candidate_point in points:
            if candidate_point != current_point:
                if (
                    next_point is None
                    or orientation(current_point, candidate_point, next_point) == 1
                ):
                    next_point = candidate_point

        current_point = next_point
        if current_point == start_point:
            break
        global execution_time
        execution_time = (timeit.default_timer() - start_time) * 1000
        print(f"Jarvis March Execution Time: {execution_time:.6f} milliseconds")
    return convex_hull_points


def brute_force(points):
    def calculate_det(a, b, c):
        return (a.x * b.y + b.x * c.y + c.x * a.y) - (a.y * b.x + b.y * c.x + c.y * a.x)

    def is_inside_polygon(point, polygon):
        n = len(polygon)
        if n < 3:
            return False

        extreme = Point(float("inf"), point.y)

        count = i = 0
        while True:
            next_i = (i + 1) % n

            if do_intersect(polygon[i], polygon[next_i], point, extreme):
                if orientation(polygon[i], point, polygon[next_i]) == 0:
                    return on_segment(polygon[i], point, polygon[next_i])

                count += 1

            i = next_i

            if i == 0:
                break

        return count % 2 == 1

    def on_segment(p, q, r):
        return (
            (q.x <= max(p.x, r.x))
            and (q.x >= min(p.x, r.x))
            and (q.y <= max(p.y, r.y))
            and (q.y >= min(p.y, r.y))
        )

    def orientation(p, q, r):
        val = (q.y - p.y) * (r.x - q.x) - (q.x - p.x) * (r.y - q.y)
        if val == 0:
            return 0  # collinear
        return 1 if val > 0 else -1  # clockwise or counterclockwise

    def do_intersect(p1, q1, p2, q2):
        o1 = orientation(p1, q1, p2)
        o2 = orientation(p1, q1, q2)
        o3 = orientation(p2, q2, p1)
        o4 = orientation(p2, q2, q1)

        if o1 != o2 and o3 != o4:
            return True

        if o1 == 0 and on_segment(p1, p2, q1):
            return True

        if o2 == 0 and on_segment(p1, q2, q1):
            return True

        if o3 == 0 and on_segment(p2, p1, q2):
            return True

        if o4 == 0 and on_segment(p2, q1, q2):
            return True

        return False

    start_time = timeit.default_timer()
    if len(points) < 3:
        return []
    sorted_points = sorted(points, key=lambda p: (p.x, p.y))
    n = len(sorted_points)
    convex_set = set()

    p0 = min(sorted_points, key=lambda point: (point.y, point.x))

    for i in range(n - 1):
        for j in range(i + 1, n):
            points_left_of_ij = points_right_of_ij = True
            for k in range(n):
                if k != i and k != j:
                    det_k = calculate_det(
                        sorted_points[i], sorted_points[j], sorted_points[k]
                    )
                    if det_k > 0:
                        points_right_of_ij = False
                    elif det_k < 0:
                        points_left_of_ij = False
                    else:
                        if (
                            sorted_points[k].x < sorted_points[i].x
                            or sorted_points[k].x > sorted_points[j].x
                            or sorted_points[k].y < sorted_points[i].y
                            or sorted_points[k].y > sorted_points[j].y
                        ):
                            points_left_of_ij = points_right_of_ij = False
                            break

            if points_left_of_ij or points_right_of_ij:
                convex_set.update([sorted_points[i], sorted_points[j]])

    sorted_convex_set = sorted(convex_set, key=lambda p: (p.x, p.y))
    sorted_convex_set = sorted(
        sorted_convex_set, key=cmp_to_key(lambda p1, p2: compare(p0, p1, p2))
    )

    convex_hull = list(sorted_convex_set)
    final_convex_hull = []

    for point in convex_hull:
        if is_inside_polygon(point, convex_hull):
            final_convex_hull.append(point)

    global execution_time
    execution_time = (timeit.default_timer() - start_time) * 1000
    print(f"Brute Force Execution Time: {execution_time:.6f} milliseconds")
    return final_convex_hull


def isInsideBoundingBox(point, bounding_box):
    x_bb, y_bb = zip(
        *[(p.x, p.y) for p in bounding_box]
    )  # Extract x and y coordinates of bounding box vertices
    cross = 0
    for i in range(4):
        x1, y1, x2, y2 = x_bb[i - 1], y_bb[i - 1], x_bb[i], y_bb[i]

        is_on_boundary = (
            point.y <= ((y2 - y1) / (x2 - x1)) * (point.x - x1) + y1
        )  # Check if the point lies on the boundary
        is_inside_segment = (
            x1 <= point.x < x2 or x2 <= point.x < x1
        ) and is_on_boundary

        if is_inside_segment:
            cross += 1

    return cross % 2 != 0


# Quick Elimination Method
def quick_elimination(points):
    def graham_scan(points):
        def polar_angle(point, pivot):
            return math.atan2(point.y - pivot.y, point.x - pivot.x)

        def graham_compare(p1, p2):
            angle1 = polar_angle(p1, pivot)
            angle2 = polar_angle(p2, pivot)

            if angle1 < angle2:
                return -1
            elif angle1 > angle2:
                return 1
            else:
                return -1 if dist_sq(pivot, p1) < dist_sq(pivot, p2) else 1

        def graham_scan_algorithm(points):
            nonlocal pivot
            n = len(points)
            if n < 3:
                return []

            pivot = min(points, key=lambda p: (p.y, p.x))
            sorted_points = sorted(points, key=cmp_to_key(graham_compare))

            stack = [pivot, sorted_points[0], sorted_points[1]]

            for i in range(2, n):
                while (
                    len(stack) > 1
                    and orientation(stack[-2], stack[-1], sorted_points[i]) != -1
                ):
                    stack.pop()
                stack.append(sorted_points[i])

            return stack

        pivot = None
        return graham_scan_algorithm(points)

    start_time = timeit.default_timer()
    candidate_points = list(points)

    # Find bounding box vertices
    leftmost = min(candidate_points, key=lambda p: p.x)
    candidate_points.remove(leftmost)
    rightmost = max(candidate_points, key=lambda p: p.x)
    candidate_points.remove(rightmost)
    topmost = max(candidate_points, key=lambda p: p.y)
    candidate_points.remove(topmost)
    bottommost = min(candidate_points, key=lambda p: p.y)
    candidate_points.remove(bottommost)

    # Correct order of bounding box vertices
    bounding_box = [topmost, leftmost, bottommost, rightmost]

    # Eliminate points inside the bounding box
    remaining_points = [
        point
        for point in candidate_points
        if not isInsideBoundingBox(point, bounding_box)
    ]

    # Include bounding box vertices in the convex hull computation
    for point in bounding_box:
        remaining_points.append(point)

    # Use a more efficient convex hull algorithm
    hull_points = graham_scan(remaining_points)
    global execution_time
    execution_time = (timeit.default_timer() - start_time) * 1000
    print(f"QE Execution Time: {execution_time:.6f} milliseconds")
    return hull_points



def monotone_chain(points):
    
    start_time = timeit.default_timer()
    if len(points) < 3:
        return

    # Sort the points lexicographically (tuples are compared lexicographically).
    # Remove duplicates to detect the case we have just one unique point.
    candidate_points = sorted(set(points), key=lambda p: (p.x, p.y))

    # Build lower hull
    lower = []
    for point in candidate_points:
        while len(lower) >= 2 and calculate_det(lower[-2], lower[-1], point) <= 0:
            lower.pop()
        lower.append(point)

    # Build upper hull
    upper = []
    for point in reversed(candidate_points):
        while len(upper) >= 2 and calculate_det(upper[-2], upper[-1], point) <= 0:
            upper.pop()
        upper.append(point)

    # Concatenation of the lower and upper hulls gives the convex hull.
    # Last point of each list is omitted because it is repeated at the beginning of the other list.
    convex_hull = lower[:-1] + upper[:-1]
    
    global execution_time
    execution_time = (timeit.default_timer() - start_time) * 1000
    print(f"Monotone Chain Execution Time: {execution_time:.6f} milliseconds")
    
    return convex_hull


class ConvexHullApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Convex Hull Visualization")
    
        self.points = []
        self.current_algorithm = None

        self.canvas = tk.Canvas(self.master, bg="lightblue")
        self.canvas.pack(expand=True, fill="both")

        button_frame = ttk.Frame(self.master)
        button_frame.pack(side=tk.TOP, pady=10)

        style = ttk.Style()
        style.configure("TButton", font=("Helvetica", 10, "bold"))

        # Add a gradient background to achieve a metallic effect
        style.map(
            "TButton",
            background=[
                ("active", "#6495ED"),
                ("pressed", "#6495ED"),
            ],  # Blue for active and pressed
            foreground=[("active", "black"), ("pressed", "black")],
        )  # Black text for active and pressed

        self.graham_button = ttk.Button(
            button_frame,
            text="Graham Scan",
            command=self.run_graham_scan,
            style="TButton",
            cursor="hand2",
        )
        self.graham_button.pack(side=tk.LEFT, padx=5)
        
        self.jarvis_button = ttk.Button(
            button_frame,
            text="Jarvis March",
            command=self.run_jarvis_march,
            style="TButton",
            cursor="hand2",
        )
        self.jarvis_button.pack(side=tk.LEFT, padx=5)
        
        self.brute_button = ttk.Button(
            button_frame,
            text="Brute Force",
            command=self.run_brute_force,
            style="TButton",
            cursor="hand2",
        )
        self.brute_button.pack(side=tk.LEFT, padx=5)
        
        self.quick_button = ttk.Button(
            button_frame,
            text="Quick Elimination X Graham Scan",
            command=self.run_quick_elimination,
            style="TButton",
            cursor="hand2",
        )
        self.quick_button.pack(side=tk.LEFT, padx=5)
        
        self.monotone_button = ttk.Button(
            button_frame,
            text="Monotone Chain",
            command=self.run_monotone_chain,
            style="TButton",
            cursor="hand2",
        )
        self.monotone_button.pack(side=tk.LEFT, padx=5)
        
        
        self.clear_button = ttk.Button(
            button_frame,
            text="Clear",
            command=self.clear_canvas,
            style="TButton",
            cursor="hand2",
        )
        self.clear_button.pack(side=tk.LEFT, padx=5)

        self.time_label = tk.Label(self.master, text="Running Time: ", bg="lightgreen")
        self.time_label.pack(pady=5)

        self.canvas.bind("<Button-1>", self.add_point)
        
        self.canvas.bind("<Motion>", self.display_coordinates)  # Bind motion event for cursor movement
        self.coordinate_label = tk.Label(self.master, text="COORDINATE_LABEL", bg="lightgreen")
        self.coordinate_label.pack(pady=5)

        self.start_time = 0

    def add_point(self, event):
        x, y = event.x, event.y
        point = Point(x, y)
        self.points.append(point)
        self.canvas.create_oval(x - 4, y - 4, x + 4, y + 4, fill="red")

    def clear_canvas(self):
        self.points = []
        self.canvas.delete("all")  # Clear canvas
        self.time_label.config(text="Running Time: ")

    def draw_points(self):
        for point in self.points:
            self.canvas.create_oval(
                point.x - 4, point.y - 4, point.x + 4, point.y + 4, fill="red"
            )

    def draw_polygon(self, polygon):
        for i in range(len(polygon) - 1):
            self.canvas.create_line(
                polygon[i].x,
                polygon[i].y,
                polygon[i + 1].x,
                polygon[i + 1].y,
                fill="black",
                width=2,
            )
        self.canvas.create_line(
            polygon[-1].x,
            polygon[-1].y,
            polygon[0].x,
            polygon[0].y,
            fill="black",
            width=3,
        )

        # Shade the area bounded by the convex hull with light green color
        self.canvas.create_polygon(
            *[coord for point in polygon for coord in (point.x, point.y)],
            fill="lightgreen",
            outline="black",
        )

        # Draw the convex hull points individually
        for point in polygon:
            self.canvas.create_oval(
                point.x - 4, point.y - 4, point.x + 4, point.y + 4, fill="blue"
            )
    
    def display_coordinates(self, event):
        x, y = event.x, event.y
        self.coordinate_label.config(text=f"{COORDINATE_LABEL} ({x}, {y})")
        
    def draw_grid(self):
        # Draw vertical gridlines
        for x in range(0, self.canvas.winfo_width(), GRID_SPACING):
            self.canvas.create_line(x, 0, x, self.canvas.winfo_height(), fill=GRID_COLOR, dash=(2, 2))

        # Draw horizontal gridlines
        for y in range(0, self.canvas.winfo_height(), GRID_SPACING):
            self.canvas.create_line(0, y, self.canvas.winfo_width(), y, fill=GRID_COLOR, dash=(2, 2))

    def run_graham_scan(self):
        self.run_algorithm("Graham Scan")

    def run_jarvis_march(self):
        self.run_algorithm("Jarvis March")

    def run_brute_force(self):
        self.run_algorithm("Brute Force")

    def run_quick_elimination(self):
        self.run_algorithm("Quick Elimination")
        
        
    def run_monotone_chain(self):
        self.run_algorithm("Monotone Chain")

    def run_algorithm(self, algorithm):
        self.canvas.delete("all")  # Clear canvas
        self.current_algorithm = algorithm

        # Start timing before calling the algorithm
        # self.start_time = timeit.default_timer()
        self.draw_grid()
        try:
            if algorithm == "Graham Scan":
                hull = graham_scan(self.points)
            elif algorithm == "Jarvis March":
                hull = jarvis_march(self.points)
            elif algorithm == "Brute Force":
                hull = brute_force(self.points)
            elif algorithm == "Quick Elimination":
                hull = quick_elimination(self.points)
            elif algorithm == "Monotone Chain":
                hull = monotone_chain(self.points)
            else:
                raise ValueError("Invalid algorithm name")
        except Exception as e:
            self.time_label.config(text=f"Error: {str(e)}")
            return

        # # Measure the algorithm's execution time only
        # execution_time = (
        #     timeit.default_timer() - self.start_time
        # ) * 1000  # Convert to milliseconds
        self.time_label.config(text=f"Running Time: {execution_time:.6f} ms")

        # Draw after timing to get accurate algorithm execution time
        self.draw_polygon(hull)
        self.draw_points()


def main():
    root = tk.Tk()
    app = ConvexHullApp(root)
    app.draw_grid()
    root.geometry("800x600")
    root.mainloop()

if __name__ == "__main__":
    main()

Graham Scan Execution Time: 0.062200 milliseconds


In [2]:
import tkinter as tk
import timeit

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def doIntersect(p1, q1, p2, q2, method):
    if method == 1:
        return do_intersect_method_1(p1, q1, p2, q2)
    elif method == 2:
        return do_intersect_method_2(p1, q1, p2, q2)
    elif method == 3:
        return do_intersect_method_3(p1, q1, p2, q2)
    else:
        return False

def do_intersect_method_1(p1, q1, p2, q2):
    def orientation(p, q, r):
        val = (float(q.y - p.y) * (r.x - q.x)) - (float(q.x - p.x) * (r.y - q.y))
        if val > 0:
            return 1  # Clockwise orientation
        elif val < 0:
            return 2  # Counterclockwise orientation
        else:
            return 0  # Collinear orientation

    def onSegment(p, q, r):
        return (
            q.x <= max(p.x, r.x)
            and q.x >= min(p.x, r.x)
            and q.y <= max(p.y, r.y)
            and q.y >= min(p.y, r.y)
        )

    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)

    if (o1 != o2) and (o3 != o4):
        return True
    if (o1 == 0) and onSegment(p1, p2, q1):
        return True
    if (o2 == 0) and onSegment(p1, q2, q1):
        return True
    if (o3 == 0) and onSegment(p2, p1, q2):
        return True
    if (o4 == 0) and onSegment(p2, q1, q2):
        return True

    return False

def do_intersect_method_2(p1, q1, p2, q2):
    slope1 = (q1.y - p1.y) / (q1.x - p1.x) if (q1.x - p1.x) != 0 else float("inf")
    slope2 = (q2.y - p2.y) / (q2.x - p2.x) if (q2.x - p2.x) != 0 else float("inf")

    if slope1 != slope2:
        intersect_x = (p2.y - p1.y + slope1 * p1.x - slope2 * p2.x) / (slope1 - slope2)
        intersect_y = slope1 * (intersect_x - p1.x) + p1.y

        if (
            min(p1.x, q1.x) <= intersect_x <= max(p1.x, q1.x)
            and min(p1.y, q1.y) <= intersect_y <= max(p1.y, q1.y)
            and min(p2.x, q2.x) <= intersect_x <= max(p2.x, q2.x)
            and min(p2.y, q2.y) <= intersect_y <= max(p2.y, q2.y)
        ):
            return True

    return False

def do_intersect_method_3(p1, q1, p2, q2):
    dx1 = q1.x - p1.x
    dy1 = q1.y - p1.y
    dx2 = q2.x - p2.x
    dy2 = q2.y - p2.y

    determinant = dx1 * dy2 - dx2 * dy1

    if determinant == 0:
        return False

    t = ((p2.x - p1.x) * dy2 - (p2.y - p1.y) * dx2) / determinant
    s = ((p2.x - p1.x) * dy1 - (p2.y - p1.y) * dx1) / determinant

    if 0 <= t <= 1 and 0 <= s <= 1:
        return True

    return False

class LineIntersectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Line Intersection")

        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        self.root.geometry(f"{int(screen_width * 0.8)}x{int(screen_height * 0.8)}")

        self.canvas = tk.Canvas(root, bg="lightgreen")
        self.canvas.pack(fill=tk.BOTH, expand=True)

        self.clicked_coordinates = []
        self.canvas.bind("<Button-1>", self.on_canvas_click)

        self.clear_button = tk.Button(
            root, text="Clear Canvas", command=self.clear_canvas, font=("Helvetica", 12)
        )
        self.clear_button.pack()

        self.clear_message_label = tk.Label(
            root, text="", font=("Helvetica", 12), bg="white"
        )
        self.clear_message_label.pack()

        self.method_var = tk.IntVar(value=1)

        self.method_frame = tk.Frame(root, bg="white")
        self.method_frame.pack(pady=10)

        tk.Radiobutton(
            self.method_frame,
            text="Orientation Method",
            variable=self.method_var,
            value=1,
            command=self.on_method_change,
            font=("Helvetica", 10),
            bg="lightgrey",
        ).grid(row=0, column=0, padx=10)
        tk.Radiobutton(
            self.method_frame,
            text="Slopes and Intercept Method",
            variable=self.method_var,
            value=2,
            command=self.on_method_change,
            font=("Helvetica", 10),
            bg="lightgrey",
        ).grid(row=0, column=1, padx=10)
        tk.Radiobutton(
            self.method_frame,
            text="Parametric Method",
            variable=self.method_var,
            value=3,
            command=self.on_method_change,
            font=("Helvetica", 10),
            bg="lightgrey",
        ).grid(row=0, column=2, padx=10)

        self.time_label = tk.Label(
            root, text="", font=("Helvetica", 12), bg="white"
        )
        self.time_label.pack()

    def on_canvas_click(self, event):
        if len(self.clicked_coordinates) < 4:
            x, y = event.x, event.y
            self.clicked_coordinates.append(Point(x, y))
            self.update_canvas()

            if len(self.clicked_coordinates) == 4:
                self.measure_time_for_methods()

    def measure_time_for_methods(self):
        method = self.method_var.get()

        p1, q1, p2, q2 = (
            self.clicked_coordinates[0],
            self.clicked_coordinates[1],
            self.clicked_coordinates[2],
            self.clicked_coordinates[3],
        )

        elapsed_time = timeit.timeit(
            lambda: doIntersect(p1, q1, p2, q2, method), number=1000
        )
        elapsed_time_ms = elapsed_time * 1000  # Convert to milliseconds

        self.time_label.config(
            text=f"Execution time for Method {method}: {elapsed_time_ms:.6f} milliseconds (avg. over 1000 runs)"
        )

    def update_canvas(self):
        self.canvas.delete("all")

        for point in self.clicked_coordinates:
            self.canvas.create_oval(
                point.x - 5, point.y - 5, point.x + 5, point.y + 5, fill="black"
            )

        if len(self.clicked_coordinates) == 4:
            p1, q1, p2, q2 = (
                self.clicked_coordinates[0],
                self.clicked_coordinates[1],
                self.clicked_coordinates[2],
                self.clicked_coordinates[3],
            )

            self.canvas.create_line(p1.x, p1.y, q1.x, q1.y, fill="blue", width=2)
            self.canvas.create_line(p2.x, p2.y, q2.x, q2.y, fill="red", width=2)

            if doIntersect(p1, q1, p2, q2, self.method_var.get()):
                self.clear_message_label.config(
                    text="Lines intersect!", fg="green"
                )
            else:
                self.clear_message_label.config(
                    text="Lines do not intersect.", fg="red"
                )

    def on_method_change(self):
        self.clear_canvas()
        self.clear_message_label.config(text="")
        self.time_label.config(text="")
        self.update_canvas()

    def clear_canvas(self):
        self.clicked_coordinates = []
        self.canvas.delete("all")
        self.clear_message_label.config(text="")
        self.time_label.config(text="")
        self.update_canvas()


if __name__ == "__main__":
    root = tk.Tk()
    app = LineIntersectionApp(root)
    root.mainloop()
